https://www.kaggle.com/code/manovirat/timeseries-using-prophet-hyperparameter-tuning

In [1]:
import pandas as pd
from prophet import Prophet
from prophet.diagnostics import cross_validation
from sklearn.model_selection import ParameterGrid
from prophet.diagnostics import performance_metrics
import random
import numpy as np
from sklearn.model_selection import TimeSeriesSplit

In [2]:
params_grid = {'seasonality_mode':('multiplicative','additive'),
               # set changepoint_prior_scale to 0.5 with steps of 0.005
               'changepoint_prior_scale':[x for x in np.arange(0.001, 0.5, 0.1)],
               'seasonality_prior_scale':[x for x in np.arange(0.01, 10, 2.5)],
              'holidays_prior_scale':[x for x in np.arange(0.01, 10, 2.5)]}

grid = ParameterGrid(params_grid)
cnt = 0
for p in grid:
    cnt = cnt+1

print('Total Possible Models',cnt)

Total Possible Models 160


In [3]:
#Define MASE Metric
def mean_absolute_scaled_error(y_true, y_pred):
    mase=0
    # Define numerator as the forecast error
    numerator = (np.abs(y_true - y_pred))

    # Define denominator as the mean absolute error of the in-sample one-step naive forecast
    y_true_ohne_1 = y_true[1:].reset_index(drop=True)
    y_true_ohne_ende = y_true[:-1].reset_index(drop=True)
    denominator = np.mean(np.abs(y_true_ohne_1 - y_true_ohne_ende))

    mase = np.mean(np.abs(numerator / denominator))

    return mase

In [4]:
# Load the data
data = pd.read_parquet('/Users/paddy/Documents/GitHub/Masterthesis_ML/data/03_032022_032023_taxi_rides.parquet')

# Convert the date column to datetime
data['date'] = pd.to_datetime(data['date'])


In [5]:
# Feature Engineering

# Add a column with the time
data['time'] = [x for x in range(0, len(data))]

data['hour_of_day'] = data['date'].dt.hour
data['day_of_week'] = data['date'].dt.dayofweek
data['day_of_month'] = data['date'].dt.day
data['month_of_year'] = data['date'].dt.month
data['year'] = data['date'].dt.year

# make a weekend column
data['weekend'] = 0
data.loc[data['day_of_week'] == 5, 'weekend'] = 1
data.loc[data['day_of_week'] == 6, 'weekend'] = 1

#make a monday column
data['monday'] = 0
data.loc[data['day_of_week'] == 0, 'monday'] = 1

#make a tuesday column
data['tuesday'] = 0
data.loc[data['day_of_week'] == 1, 'tuesday'] = 1

#make a wednesday column
data['wednesday'] = 0
data.loc[data['day_of_week'] == 2, 'wednesday'] = 1

#make a thursday column
data['thursday'] = 0
data.loc[data['day_of_week'] == 3, 'thursday'] = 1

#make a friday column
data['friday'] = 0
data.loc[data['day_of_week'] == 4, 'friday'] = 1

#make a saturday column
data['saturday'] = 0
data.loc[data['day_of_week'] == 5, 'saturday'] = 1

#make a sunday column
data['sunday'] = 0
data.loc[data['day_of_week'] == 6, 'sunday'] = 1

# Drop the first three rows
data = data.dropna().reset_index(drop=True)

# convert date column to datetime without timezone
data['date'] = pd.to_datetime(data['date'],format='%d/%m/%y %H:%M:%S').dt.strftime('%Y-%m-%d %H:%M:%S') 

# Rename column count to y
data = data.rename(columns={'count': 'y'})
data = data.rename(columns={'date': 'ds'})


train_data = data

In [6]:
cv_folds = TimeSeriesSplit(n_splits=5)

#convert data to only date not time
cv_data = train_data.copy()

#convert to timestamp with date format
#cv_data.ds = pd.to_datetime(cv_data.ds).dt.date

print(cv_data.ds)   

#Define cutoff dates for cross validation at end of train_data/cv_folds
cutoffs = []
# for train_index, test_index in cv_folds.split(train_data):
#     cutoffs.append(train_data.ds.iloc[test_index].max())
#     #convert to timestamp
#     cutoffs = pd.to_datetime(cutoffs)
    
# print(cutoffs)

for i, (train_index, test_index) in enumerate(cv_folds.split(cv_data)):
    print(f"Fold {i}:")
    print(f"  Train: index={train_index}")
    print(f"  Test:  index={test_index}")
    #get the date for the last train index
    cutoffs.append(cv_data.ds.iloc[train_index].max())
    #only date not time
    #cutoffs = pd.to_datetime(cutoffs).date

0       2022-04-01 00:00:00
1       2022-04-01 01:00:00
2       2022-04-01 02:00:00
3       2022-04-01 03:00:00
4       2022-04-01 04:00:00
               ...         
8755    2023-03-31 19:00:00
8756    2023-03-31 20:00:00
8757    2023-03-31 21:00:00
8758    2023-03-31 22:00:00
8759    2023-03-31 23:00:00
Name: ds, Length: 8760, dtype: object
Fold 0:
  Train: index=[   0    1    2 ... 1457 1458 1459]
  Test:  index=[1460 1461 1462 ... 2917 2918 2919]
Fold 1:
  Train: index=[   0    1    2 ... 2917 2918 2919]
  Test:  index=[2920 2921 2922 ... 4377 4378 4379]
Fold 2:
  Train: index=[   0    1    2 ... 4377 4378 4379]
  Test:  index=[4380 4381 4382 ... 5837 5838 5839]
Fold 3:
  Train: index=[   0    1    2 ... 5837 5838 5839]
  Test:  index=[5840 5841 5842 ... 7297 7298 7299]
Fold 4:
  Train: index=[   0    1    2 ... 7297 7298 7299]
  Test:  index=[7300 7301 7302 ... 8757 8758 8759]


In [7]:
MASE_list = []
best_mase = 100000000
best_params = None
for p in grid:
    train = pd.DataFrame(train_data)
    test = pd.DataFrame()
    print(p)
    random.seed(0)
    train_model =Prophet(changepoint_prior_scale = p['changepoint_prior_scale'],
                        holidays_prior_scale = p['holidays_prior_scale'],
                        seasonality_prior_scale = p['seasonality_prior_scale'],
                        seasonality_mode = p['seasonality_mode'],
                        weekly_seasonality=True,
                        daily_seasonality = True,
                        yearly_seasonality = False,
                        )
    
    # add regressors
    train_model.add_regressor('hour_of_day')
    train_model.add_regressor('day_of_week')
    train_model.add_regressor('day_of_month')
    train_model.add_regressor('month_of_year')
    train_model.add_regressor('year')
    train_model.add_regressor('weekend')
    train_model.add_regressor('monday')
    train_model.add_regressor('tuesday')
    train_model.add_regressor('wednesday')
    train_model.add_regressor('thursday')
    train_model.add_regressor('friday')
    train_model.add_regressor('saturday')
    train_model.add_regressor('sunday')
    train_model.add_regressor('time')


    train_model.fit(train_data)
    
    df_cv = cross_validation(train_model, period='60 days', horizon='30 days', parallel="processes")

     # X and y to pandas dataframe
    X_train = pd.DataFrame(df_cv)

    y_true = df_cv['y'].values
    #make y_true as dataframe
    y_true = pd.DataFrame(y_true)

    y_pred = df_cv['yhat'].values
    #make y_pred as dataframe
    y_pred = pd.DataFrame(y_pred)

    MASE_list.append(mean_absolute_scaled_error(y_true, y_pred))
    


    # print('RMSE:',np.sqrt(MSE))
    # RMSE_list.append(np.sqrt(MSE))

# Find the best parameters
tuning_results = pd.DataFrame(grid)
tuning_results['mase'] = MASE_list
print(tuning_results)

{'changepoint_prior_scale': 0.001, 'holidays_prior_scale': 0.01, 'seasonality_mode': 'multiplicative', 'seasonality_prior_scale': 0.01}


15:26:28 - cmdstanpy - INFO - Chain [1] start processing
15:26:28 - cmdstanpy - INFO - Chain [1] done processing
15:26:29 - cmdstanpy - INFO - Chain [1] start processing
15:26:29 - cmdstanpy - INFO - Chain [1] start processing
15:26:30 - cmdstanpy - INFO - Chain [1] start processing
15:26:30 - cmdstanpy - INFO - Chain [1] done processing
15:26:30 - cmdstanpy - INFO - Chain [1] start processing
15:26:30 - cmdstanpy - INFO - Chain [1] start processing
15:26:30 - cmdstanpy - INFO - Chain [1] done processing
15:26:30 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimization terminated abnormally. Falling back to Newton.
15:26:30 - cmdstanpy - INFO - Chain [1] done processing
15:26:30 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimization terminated abnormally. Falling back to Newton.
15:26:30 - cmdstanpy - INFO - Chain [1] done processing
15:26:30 - cmdstanpy - INFO - Chain [1] start processing
15:26:30 

{'changepoint_prior_scale': 0.001, 'holidays_prior_scale': 0.01, 'seasonality_mode': 'multiplicative', 'seasonality_prior_scale': 2.51}


15:27:43 - cmdstanpy - INFO - Chain [1] start processing
15:27:44 - cmdstanpy - INFO - Chain [1] done processing
15:27:45 - cmdstanpy - INFO - Chain [1] start processing
15:27:45 - cmdstanpy - INFO - Chain [1] start processing
15:27:45 - cmdstanpy - INFO - Chain [1] start processing
15:27:45 - cmdstanpy - INFO - Chain [1] done processing
15:27:45 - cmdstanpy - INFO - Chain [1] start processing
15:27:45 - cmdstanpy - INFO - Chain [1] start processing
15:27:45 - cmdstanpy - INFO - Chain [1] done processing
15:27:45 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimization terminated abnormally. Falling back to Newton.
15:27:45 - cmdstanpy - INFO - Chain [1] done processing
15:27:45 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimization terminated abnormally. Falling back to Newton.
15:27:46 - cmdstanpy - INFO - Chain [1] done processing
15:27:46 - cmdstanpy - INFO - Chain [1] start processing
15:27:46 

{'changepoint_prior_scale': 0.001, 'holidays_prior_scale': 0.01, 'seasonality_mode': 'multiplicative', 'seasonality_prior_scale': 5.01}


...

15:54:48 - cmdstanpy - INFO - Chain [1] start processing
15:54:50 - cmdstanpy - INFO - Chain [1] done processing
15:54:51 - cmdstanpy - INFO - Chain [1] start processing
15:54:51 - cmdstanpy - INFO - Chain [1] done processing
15:54:51 - cmdstanpy - INFO - Chain [1] start processing
15:54:51 - cmdstanpy - INFO - Chain [1] done processing
15:54:51 - cmdstanpy - INFO - Chain [1] start processing
15:54:51 - cmdstanpy - INFO - Chain [1] start processing
15:54:52 - cmdstanpy - INFO - Chain [1] start processing
15:54:52 - cmdstanpy - INFO - Chain [1] done processing
15:54:52 - cmdstanpy - INFO - Chain [1] done processing
15:54:53 - cmdstanpy - INFO - Chain [1] done processing


{'changepoint_prior_scale': 0.401, 'holidays_prior_scale': 7.51, 'seasonality_mode': 'additive', 'seasonality_prior_scale': 7.51}


15:54:54 - cmdstanpy - INFO - Chain [1] start processing
15:54:56 - cmdstanpy - INFO - Chain [1] done processing
15:54:57 - cmdstanpy - INFO - Chain [1] start processing
15:54:57 - cmdstanpy - INFO - Chain [1] start processing
15:54:57 - cmdstanpy - INFO - Chain [1] done processing
15:54:57 - cmdstanpy - INFO - Chain [1] start processing
15:54:57 - cmdstanpy - INFO - Chain [1] done processing
15:54:57 - cmdstanpy - INFO - Chain [1] start processing
15:54:58 - cmdstanpy - INFO - Chain [1] start processing
15:54:58 - cmdstanpy - INFO - Chain [1] done processing
15:54:58 - cmdstanpy - INFO - Chain [1] done processing


     changepoint_prior_scale  holidays_prior_scale seasonality_mode  \
0                      0.001                  0.01   multiplicative   
1                      0.001                  0.01   multiplicative   
2                      0.001                  0.01   multiplicative   
3                      0.001                  0.01   multiplicative   
4                      0.001                  0.01         additive   
..                       ...                   ...              ...   
155                    0.401                  7.51   multiplicative   
156                    0.401                  7.51         additive   
157                    0.401                  7.51         additive   
158                    0.401                  7.51         additive   
159                    0.401                  7.51         additive   

     seasonality_prior_scale      mase  
0                       0.01  0.856370  
1                       2.51  0.790903  
2                       

15:55:00 - cmdstanpy - INFO - Chain [1] done processing


In [8]:
best_params = grid[np.argmin(MASE_list)]
print('Best Parameters',best_params)
print('Best MASE',np.min(MASE_list))
print(MASE_list)

Best Parameters {'seasonality_prior_scale': 2.51, 'seasonality_mode': 'multiplicative', 'holidays_prior_scale': 0.01, 'changepoint_prior_scale': 0.201}
Best MASE 0.761220597049664
[0.8563698257725282, 0.7909027688420335, 0.7858926376203352, 0.7889209399004373, 0.9511363293132634, 0.95437334599337, 0.9533659674478803, 0.9500610942966798, 7.580393109236279, 0.8151192025465371, 0.804066996707013, 0.8453324107177499, 7.5032357061703205, 7.498384567314259, 7.497961721894802, 7.49774597792881, 7.564819643579489, 0.8634200388358726, 0.8206286762134277, 0.8072198640978504, 1.0081615352904727, 7.66273695861852, 7.663032617241574, 7.66308787072322, 7.580018311438854, 0.8877799841775548, 1.0473071744746187, 0.8014824256651529, 7.734594605499037, 7.694969949873044, 7.694848146938336, 7.695249393520143, 0.8722983203035982, 0.7979686062495993, 0.8312639074642193, 0.8000211864038174, 0.7820956911900737, 0.7889589182654203, 0.7882613430830369, 0.7818763321605173, 0.84292451010811, 0.8175879476210495, 